In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import selenium


In [5]:
url = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

In [ ]:
response = requests.get(url)
response.status_code
soup = BeautifulSoup(response.text, "html.parser")
headings = soup.find_all("h2")
nombre_primer_hotel = headings[0].get_text().strip().split("\n")[0]
estrellas_primer_hotel = headings[0].get_text().strip().split("\n")[1].strip()

In [16]:
dict_hoteles = {
    "nombres_hotel": [],
    "estrellas_hotel": []
}

In [17]:
dict_hoteles

{'nombres_hotel': [], 'estrellas_hotel': []}

In [18]:
for head in headings:
    nombre_hotel = head.get_text().strip().split("\n")[0]
    dict_hoteles["nombres_hotel"].append(nombre_hotel)    

In [19]:
for head in headings:
    estrella_hotel = head.get_text().strip().split("\n")[1].strip()
    dict_hoteles["estrellas_hotel"].append(estrella_hotel)

In [24]:
dict_hoteles

{'nombres_hotel': ['ibis Styles Madrid Prado',
  'Mercure Madrid Centro',
  'ibis budget Madrid Centro Lavapies',
  'ibis Styles Madrid Centro Maravillas',
  'Novotel Madrid Center',
  'ibis budget Madrid Calle 30',
  'ibis Madrid Centro las Ventas',
  'ibis budget Madrid Centro las Ventas',
  'Novotel Madrid City Las Ventas',
  'ibis Styles Madrid City Las Ventas'],
 'estrellas_hotel': ['3 Estrellas',
  '4 Estrellas',
  '1 Estrellas',
  '2 Estrellas',
  '4 Estrellas',
  '1 Estrellas',
  '2 Estrellas',
  '1 Estrellas',
  '4 Estrellas',
  '4 Estrellas']}

In [11]:
def scrap_hoteles(url):
    dict_hoteles = {
        "nombres_hotel": [],
        "estrellas_hotel": []
    }
    res = requests.get(url)
    if res.status_code == 200:
        sopa_hoteles = BeautifulSoup(res.text, "html.parser")
        headings = sopa_hoteles.find_all("h2")
        for head in headings:
            nombres_hotel = head.get_text().strip().split("\n")[0]
            estrellas_hotel = head.get_text().strip().split("\n")[1].strip()
            dict_hoteles["nombres_hotel"].append(nombres_hotel)
            dict_hoteles["estrellas_hotel"].append(estrellas_hotel)
    else:
        return f"No se ha podido encontrar la url, status code: {res.status_code}"
    return dict_hoteles


In [ ]:
def scrap_hoteles(url):
    """
    Función para extraer información sobre hoteles desde una URL utilizando web scraping con BeautifulSoup.

    Esta función realiza una solicitud HTTP a la URL proporcionada, obtiene el contenido HTML de la página, y extrae
    los nombres de los hoteles y sus estrellas desde los elementos `<h2>` que contienen esta información.
    La función asume que el texto en cada `<h2>` está separado por un salto de línea (`\n`), donde la primera parte es 
    el nombre del hotel y la segunda parte es la cantidad de estrellas del hotel.

    Parámetros:
    ----------
    url : str
        La URL de la página web desde la cual se desea hacer scraping.

    Retorna:
    --------
    dict_hoteles : dict
        Un diccionario que contiene dos claves:
        - 'nombres_hotel': una lista con los nombres de los hoteles extraídos de la página.
        - 'estrellas_hotel': una lista con la cantidad de estrellas de cada hotel.
    
    Si la solicitud HTTP falla (código de estado diferente a 200), la función retorna un mensaje de error.

    Ejemplo:
    --------
    url = "https://ejemplo.com/hoteles"
    resultado = scrap_hoteles(url)
    
    print(resultado)
    # {'nombres_hotel': ['Mercure Madrid Centro', 'Hotel XYZ'], 'estrellas_hotel': ['4 Estrellas', '5 Estrellas']}
    """
    dict_hoteles = {
        "nombres_hotel": [],
        "estrellas_hotel": []
    }
    
    # Realizar solicitud HTTP
    res = requests.get(url)
    
    # Verificar que la solicitud fue exitosa (código 200)
    if res.status_code == 200:
        sopa_hoteles = BeautifulSoup(res.text, "html.parser")
        
        # Buscar todos los elementos <h2> que contienen la información de los hoteles
        headings = sopa_hoteles.find_all("h2")
        
        for head in headings:
            # Obtener el texto y eliminar espacios al principio y final
            text = head.get_text().strip()
            parts = text.split("\n")
            
            # Manejar el caso donde hay menos de dos partes
            if len(parts) >= 2:
                nombres_hotel = parts[0].strip()  # El nombre del hotel es la primera parte
                estrellas_hotel = parts[1].strip()  # La cantidad de estrellas es la segunda parte
            else:
                nombres_hotel = parts[0].strip()  # Si solo hay el nombre del hotel
                estrellas_hotel = "No rating available"  # Asignar valor por defecto para la estrella si falta
            
            # Añadir los datos al diccionario
            dict_hoteles["nombres_hotel"].append(nombres_hotel)
            dict_hoteles["estrellas_hotel"].append(estrellas_hotel)
    
    else:
        return f"No se ha podido encontrar la url, status code: {res.status_code}"
    
    return dict_hoteles
    

In [12]:
dict_hoteles = scrap_hoteles(url)
dict_hoteles

{'nombres_hotel': ['ibis Styles Madrid Prado',
  'Mercure Madrid Centro',
  'ibis budget Madrid Centro Lavapies',
  'ibis Styles Madrid Centro Maravillas',
  'Novotel Madrid Center',
  'ibis budget Madrid Calle 30',
  'ibis Madrid Centro las Ventas',
  'ibis budget Madrid Centro las Ventas',
  'Novotel Madrid City Las Ventas',
  'ibis Styles Madrid City Las Ventas'],
 'estrellas_hotel': ['3 Estrellas',
  '4 Estrellas',
  '1 Estrellas',
  '2 Estrellas',
  '4 Estrellas',
  '1 Estrellas',
  '2 Estrellas',
  '1 Estrellas',
  '4 Estrellas',
  '4 Estrellas']}

In [16]:
df_hoteles_bs = pd.DataFrame(dict_hoteles)
df_hoteles_bs["id_hotel"] = 0
df_hoteles_bs

,nombres_hotel,estrellas_hotel,id_hotel
0,ibis Styles Madrid Prado,3 Estrellas,0
1,Mercure Madrid Centro,4 Estrellas,0
2,ibis budget Madrid Centro Lavapies,1 Estrellas,0
3,ibis Styles Madrid Centro Maravillas,2 Estrellas,0
4,Novotel Madrid Center,4 Estrellas,0
5,ibis budget Madrid Calle 30,1 Estrellas,0
6,ibis Madrid Centro las Ventas,2 Estrellas,0
7,ibis budget Madrid Centro las Ventas,1 Estrellas,0
8,Novotel Madrid City Las Ventas,4 Estrellas,0
9,ibis Styles Madrid City Las Ventas,4 Estrellas,0
